In [1]:
# Define environment variables 
%env DEEP_ROOT=/content/drive/My Drive/brAIn-Power
%env DEEP_DATA=/content/drive/My Drive/correct_data

# Connect to drive
from google.colab import drive
drive.mount('/content/drive')

# Change directory before importing files
import os
deep_root = os.getenv("DEEP_ROOT")
%cd $deep_root

!pip install import-ipynb
import import_ipynb

env: DEEP_ROOT=/content/drive/My Drive/brAIn-Power
env: DEEP_DATA=/content/drive/My Drive/correct_data
Mounted at /content/drive
/content/drive/My Drive/brAIn-Power
  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2975 sha256=16ec6312b0f5f2d15b3ffc1759439d68b2b343821527fae7ecdb989926ca1087
  Stored in directory: /root/.cache/pip/wheels/b1/5e/dc/79780689896a056199b0b9f24471e3ee184fbd816df355d5f0
Successfully built import-ipynb


In [2]:
from LoadData import CustomDataset, load_data_2np,one_hot_encode
import UNet
import torch
import time
import numpy as np
from torch.utils.data import DataLoader,Dataset

importing Jupyter notebook from LoadData.ipynb
importing Jupyter notebook from UNet.ipynb


In [3]:
# Define some helper functions for later:
def IoU_score(target, prediction):
  iou_score = 0
  for i in range(target.shape[0]):
    intersection = np.logical_and(target[i], prediction[i])
    union = np.logical_or(target[i], prediction[i])
    iou_score += np.sum(intersection[i]) / np.sum(union[i])
  iou_score = iou_score/target.shape[0]
  return iou_score

In [4]:
# Load data the first time
#X_train, Y_train, X_valid, Y_valid, X_trans, Y_trans, X_test, Y_test = load_data_2np(hot_encoding=True, valid_perc=0.1, train_perc=0.1,show=0,CAD_perc=0,background=True)

In [5]:
Valid_perc = 0.1

X_train = np.load(f"data/compressed_data/without_background/X_train,.npy")
Y_train = np.load(f"data/compressed_data/without_background/Y_train,.npy")
X_test = np.load(f"data/compressed_data/without_background/X_test,.npy")
Y_test = np.load(f"data/compressed_data/without_background/Y_test,.npy")

Y_train = one_hot_encode(imgs=Y_train)
Y_test = one_hot_encode(imgs=Y_test)

idx = int(Valid_perc*X_train.shape[0])
X_valid = X_train[0:idx]
X_train = X_train[idx::]

Y_valid = Y_train[0:idx]
Y_train = Y_train[idx::]

In [6]:
# Save data for easier load next time - remember to name accodingly
# The naming scheme refers pparameters of the load_data_2np function
#np.save("data/X_train_hot_v0.1_t0.1_s0_c0_True", X_train)
#np.save("data/Y_train_hot_v0.1_t0.1_s0_c0_True", Y_train)
#np.save("data/X_valid_hot_v0.1_t0.1_s0_c0_True", X_valid)
#np.save("data/Y_valid_hot_v0.1_t0.1_s0_c0_True", Y_valid)
#np.save("data/X_trans_hot_v0.1_t0.1_s0_c0_True", X_trans)
#np.save("data/Y_trans_hot_v0.1_t0.1_s0_c0_True", Y_trans)
#np.save("data/X_test_hot_v0.1_t0.1_s0_c0_True", X_test)
#np.save("data/Y_test_hot_v0.1_t0.1_s0_c0_True", Y_test)

In [7]:
# # Load dataset from .npy files
# data_set = "hot_v0.1_t0.1_s0_c0_True"

# X_train = np.load(f"data/X_train_{data_set}.npy")
# Y_train = np.load(f"data/Y_train_{data_set}.npy")
# X_valid = np.load(f"data/X_valid_{data_set}.npy")
# Y_valid = np.load(f"data/Y_valid_{data_set}.npy")
# X_test = np.load(f"data/X_test_{data_set}.npy")
# Y_test = np.load(f"data/Y_test_{data_set}.npy")



In [8]:
# Load transformations if needed
#X_trans = np.load(f"data/X_trans_{data_set}.npy")
#Y_trans = np.load(f"data/Y_trans_{data_set}.npy")

In [9]:
DS_train = CustomDataset(X_train, Y_train, one_hot=True)
DS_valid = CustomDataset(X_valid, Y_valid, one_hot=True)
DS_test = CustomDataset(X_test, Y_test, one_hot=True)

In [10]:
# Clear space
del X_train
del Y_train
del X_valid
del Y_valid
del X_test
del Y_test

In [11]:
# Perform preprocessing
# DS_train.gray_gamma_enhanced()
# DS_valid.gray_gamma_enhanced()
# DS_test.gray_gamma_enhanced()

In [12]:
# Check if GPU is available and functions to convert Tensors to Cuda
use_cuda = torch.cuda.is_available()
print("Running GPU.") if use_cuda else print("No GPU available.")

Running GPU.


In [19]:
# Load in model
%%capture
net = UNet.UNet18(n_classes=9, rgb=True)
net = net.to(torch.half)
if use_cuda:
    print('converting network to cuda-enabled')
    net.cuda()


In [20]:
# Converts tensors to cuda, if available
def get_variable(x):
    if use_cuda:
        return x.cuda()
    return x

In [21]:
# Get numpy array for both cuda and not
def get_numpy(x):
    if use_cuda:
        return x.cpu().data.numpy()
    return x.data.numpy()

In [22]:
# Create lossfunction and optimizer
criterion = UNet.DiceLoss()
optimizer = torch.optim.Adam(net.parameters(),lr=1e-4)

In [23]:
# Define hyper parameters
NUM_EPOCHS = 50
check_at = 2

In [ ]:
# Define list to stor intermediat results
valid_iter = []
valid_loss = []
valid_iou = []

train_iter = []
train_loss = []
train_iou = []

# Define data loaders to serve data
DL_train = DataLoader(DS_train, batch_size=4, shuffle=True)
DL_valid = DataLoader(DS_valid)

# Training loop
net.train()
for epoch in range(NUM_EPOCHS):
    print("train")
    for i, data in enumerate(DL_train, 0):
        net.train()
        input = data[0].type(torch.HalfTensor)
        target = data[1].type(torch.HalfTensor)

        # Tranfer to GPU if possible
        input = get_variable(input)
        target = get_variable(target)

        # Train the network
        optimizer.zero_grad()
        output = net(input)
        loss = criterion(output,target)
        loss.backward()
        optimizer.step()

        # Save space
        del input
        del target
        del output
        del loss

    # If we want to evaluate
    if epoch % check_at == 0:
      net.eval()

      with torch.no_grad():
        # Evaluate training
        train_losses, train_ious, train_batches = 0, 0, 0
        for i, data in enumerate(DL_train, 0):
          input = data[0].type(torch.HalfTensor)
          target = data[1].type(torch.HalfTensor)

          # Tranfer to GPU if possible
          input = get_variable(input)
          target = get_variable(target)
          
          output = net(input)

          # Record performance
          train_batches += 1
          train_losses += criterion(output, target).item()
          train_ious += IoU_score(get_numpy(target), get_numpy(output))

          # Save space
          del input
          del target
          del output

        train_iter.append(epoch)
        train_loss.append(train_losses / train_batches)
        train_iou.append(train_ious / train_batches)

        # Evaluate validation
        val_losses, val_ious, val_batches = 0, 0, 0
        for i, valid_data in enumerate(DL_valid, 0):
            input = valid_data[0].type(torch.HalfTensor)
            target = valid_data[1].type(torch.HalfTensor)
            
            # Tranfer to GPU if possible
            input = get_variable(input)
            target = get_variable(target)
            
            output = net(input)

            # Record performance
            val_batches += 1
            val_losses += criterion(output, target).item()
            val_ious += IoU_score(get_numpy(target), get_numpy(output))

            # Save space
            del input
            del target
            del output

        valid_iter.append(epoch)
        valid_loss.append(val_losses / val_batches)
        valid_iou.append(val_ious / val_batches)

        print(f'At epoch {epoch} Training loss is at {train_loss[-1]} and the IoU is at {train_iou[-1]}%')
        print(f'At epoch {epoch} Validation loss is at {valid_loss[-1]} and the IoU is at {valid_iou[-1]}%')

train
At epoch 0 Training loss is at nan and the IoU is at 0.22323358058929443%
At epoch 0 Validation loss is at nan and the IoU is at 0.7901141826923077%
train
train


In [ ]:
net.eval()
input = X_train[19].reshape([1,3,256,256])
input = get_variable(torch.tensor(input))
output = net(input)
from LoadData import one_hot2_2d
import matplotlib.pyplot as plt

img_label = one_hot2_2d(get_numpy(output)[0])
img_pred = one_hot2_2d(Y_train[19])

plt.subplot(131)
plt.imshow(img_label[0])

plt.subplot(132)
plt.imshow(img_pred[0])

plt.subplot(133)
plt.imshow(X_train[19].transpose([2,1,0]))
plt.show()

print(X_train.shape)